# Dati Anac + Banca dati Servizio Contratti Pubblici - SCP
## Data processing

Il risultante di questo notebook è un dataset che processa i dati provenienti dalla Banca dati Servizio Contratti Pubblici relativi ad un certo indice PA congiunti con i dati estratti dal db di ANAC.


### Banca dati Servizio Contratti Pubblici - SCP

Il Dataset contiene :

- gli avvisi
- i bandi
- gli esiti di gara 

in formato aperto raccolti dalla Banca dati SCP - Servizio Contratti Pubblici gestita dalla Direzione Generale per la regolazione e i contratti pubblici del Ministero delle Infrastrutture e Trasporti. 

Il Servizio Contratti Pubblici (SCP) consente la pubblicazione on line di bandi di gara, avvisi sui risultati delle procedure di affidamento e programmi triennali di opere pubbliche. 

Maggiori informazioni sono disponibili sul sito www.serviziocontrattipubblici.it che consente alle stazioni appaltanti di pubblicare gli avvisi ed i bandi per lavori, servizi e forniture ed altre informazioni. 

Servizio Contratti Pubblici è una delle banche dati nazionali a titolarità del Ministero delle Infrastrutture e Trasporti elencate nell'allegato B del D.Lgs n.97/2016 (art. 9) che modifica l’obbligo di pubblicazione previsto nella normativa sulla trasparenza D.Lgs n.33/2013 (art. 9bis). I dati sono aggiornati quotidianamente.
http://dati.mit.gov.it/catalog/dataset/scp

https://www.serviziocontrattipubblici.it

https://contrattipubblici.org

## General

In [ ]:
#IPython setting
import pandas as pd
import numpy as np
import json 
import os
import re
import seaborn as sns
import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 
import matplotlib.style as style
import seaborn as sns 
%matplotlib inline 

from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #setta l'esecuzione di più statement in unica cella
pd.set_option('display.max_colwidth', -1) 
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60) 

# Set up formatting so larger numbers aren't displayed in scientific notation (h/t @thecapacity)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set_context("notebook")
sns.set()
sns.set_style("darkgrid")
sns.set_palette(sns.color_palette("Set2", 10))
style.use('fivethirtyeight')

### Filtro su una determinata struttura 

### http://www.indicepa.gov.it

[INDICE PA Comune di Torino](http://www.indicepa.gov.it/elenco-cod-fisc/n-elenco-cf.php?cod_amm=c_l219)

**CITTA' DI TORINO - APPALTI LAVORI PUBBLICI - indice PA: 00514490010**

In [ ]:
filtroCfStrutturaProponente="00514490010"

## Dati ANAC

In [ ]:
anac = pd.read_csv("data/anacDataset_postprocessed.tsv", sep='\t', low_memory=False)

In [ ]:
list(anac)

In [ ]:
anac.oggetto.value_counts()

In [ ]:
anac.denominazionePrimoaggiudicatario.value_counts()

In [ ]:
anac.denominazioneStrutturaProponente.value_counts()

In [ ]:
anac.jsonPartecipanti

In [ ]:
#seziono su stazione appaltante di torino
anac_to = anac[anac.cfStrutturaProponente==filtroCfStrutturaProponente]

In [ ]:
anac_to.head(3)

# Dati SCP


In [ ]:
avvisi = pd.read_csv("data/ServizioContrattiPubblici/scpavvisi.csv")
bandi = pd.read_csv("data/ServizioContrattiPubblici/scpbandi.csv")
esiti = pd.read_csv("data/ServizioContrattiPubblici/scpesiti.csv")

In [ ]:
##ex: '"2009-11-19T23:00:00.000Z"'
d_format = '%Y-%m-%dT%H:%M:%S.%f'

In [ ]:
avvisi_data_labels = ["data_pubb_esito_scp","data_pubb_esito_guue", "data_pubb_guri", "data_pubb_albo", 'data_agg_def',"termine_pres_dom_off", 'data_pres_fine']
bandi_data_labels = ["data_pubb_bando_scp","data_pubb_guue", "data_pubb_guri", "data_pubb_albo", "termine_pres_dom_off"]
esito_data_labels = ["data_pubb_esito_scp","data_pubb_esito_guue","data_pubb_guri","data_pubb_albo","data_agg_def", "data_pres_fine", "termine_pres_dom_off"]

### Preprocessing fix malformatted fields and Data

In [ ]:
#Molti campi hanno il "", tipo codice_fiscale_stazione_appaltante o le varie date
def cleanDoubleQuotes(x):
    # if x is a string,
    if type(x) is str:
        last = x.replace('"', '')
        return last
    else:
        return

avvisi= avvisi.applymap(cleanDoubleQuotes)
bandi = bandi.applymap(cleanDoubleQuotes)
esiti = esiti.applymap(cleanDoubleQuotes)

In [ ]:
d_format = '%Y-%m-%dT%H:%M:%S.%f'
#If ‘coerce’, then invalid parsing will be set as NaN
for l in avvisi_data_labels:
    avvisi[l]= pd.to_datetime(avvisi[l], format=d_format, errors='coerce')
    avvisi[l]= avvisi[l].dt.date
#If ‘coerce’, then invalid parsing will be set as NaN
for l in bandi_data_labels:
    bandi[l]= pd.to_datetime(bandi[l], format=d_format, errors='coerce')
    bandi[l]= bandi[l].dt.date
#If ‘coerce’, then invalid parsing will be set as NaN
for l in esito_data_labels:
    esiti[l]= pd.to_datetime(esiti[l], format=d_format, errors='coerce')    
    esiti[l]= esiti[l].dt.date

In [ ]:
bandi.cpv.isnull().sum()

In [ ]:
len(avvisi)
len(bandi)
len(esiti)

In [ ]:
avvisi.to_csv("data/ServizioContrattiPubblici/processed/avvisi.csv",index=False)

In [ ]:
esiti.to_csv("data/ServizioContrattiPubblici/processed/esiti.csv",index=False)

In [ ]:
bandi.to_csv("data/ServizioContrattiPubblici/processed/bandi.csv",index=False)

### Filtro dataset su valore impostato 

In [ ]:
avvisi_to = avvisi[avvisi.codice_fiscale_stazione_appaltante==filtroCfStrutturaProponente]
bandi_to = bandi[bandi.codice_fiscale_stazione_appaltante==filtroCfStrutturaProponente]
esiti_to = esiti[esiti.codice_fiscale_stazione_appaltante==filtroCfStrutturaProponente]

In [ ]:
len(avvisi_to)
len(bandi_to)
len(esiti_to)
esiti_to.cig.isnull().sum()

In [ ]:
#bandi per anno
#bandi_to.data_pubb_albo.dt.year.value_counts()

In [ ]:
#dire che si è cercato agganci con opencup ma ci sono troppo record senza cup
bandi_to.cup.isnull().sum()

In [ ]:
bandi_esiti_to = pd.merge(bandi_to, esiti_to, on='cig', how='left')

In [ ]:
bandi_esiti_to.shape

In [ ]:
z_bandi_esiti_to = pd.merge(esiti_to, bandi_to, on='cig', how='left')
z_bandi_esiti_to.shape

## JOIN ANAC e ESITI

In questo caso voglio estratte un csv finale dato da tutti gli esiti di gara presenti per la PA scelta e gli eventuali dati presenti per i CIG elencati provenienti dal dataset di ANAC

In [ ]:
#bandi_anac_TO = pd.merge(bandi_to,anac_to, on='cig', how='left')

In [ ]:
esiti_anac_TO = pd.merge(esiti_to,anac_to, on='cig', how='left')
len(esiti_anac_TO)
len(esiti_to)

In [ ]:
#esiti_anac_TO.dtypes
esiti_anac_TO.jsonPartecipanti.isnull().sum()

In [ ]:
esiti_anac_TO.dtypes

In [ ]:
esiti_anac_TO.denominazione_stazione_appaltante.value_counts()

In [ ]:
esiti_anac_TO.luogo_esecuzione.value_counts()

In [ ]:
esiti_anac_TO.tipo_evento.value_counts()

In [ ]:
esiti_anac_TO.tipo_bando.value_counts()

In [ ]:
esiti_anac_TO.tipo_procedura.value_counts()

In [ ]:
esiti_anac_TO.categoria.value_counts()

In [ ]:
esiti_anac_TO.criterio_aggiudicazione.value_counts()

In [ ]:
esiti_anac_TO.tipo_settore.value_counts()

In [ ]:
esiti_anac_TO.sceltaContraente.value_counts()

In [ ]:
esiti_anac_TO.to_csv("Esiti_anac_"+filtroCfStrutturaProponente+".csv", index=False, sep=";")

## EDA

In [ ]:
ax = sns.countplot(x=esiti_anac_TO.data_pubb_albo.dt.year, data=esiti_anac_TO)
ax.set_title("Numero di esiti per anno (anno pubblicazione albo)");
ax.set_ylabel("Numero di esiti");
ax.set_xlabel("Anno");

In [ ]:
#bandi per tipologia di intervento
ax = sns.countplot(x=esiti_anac_TO.data_pubb_albo.dt.year, data=esiti_anac_TO,  hue=esiti_anac_TO.categoria)
ax.set_title("Esiti per anno e tipologia di intervento");
ax.set_ylabel("Numero di esiti");
ax.set_xlabel("Anno");

In [ ]:
#band per fascia di importo
#bandi per anno: numero e importo
#Numeri Bandi di gara per tipologia: 
#Importi Bandi di gara per tipologia:

### OpenCUP

I record ai quali è stato assegnato un cup sono molto pochi, non ha quindi rilevanza nello stato attuale il join con il dataset OPENCUP

In [ ]:
cup = pd.read_csv("data/cup/AREA_NORD_EST_CSV.csv", sep='|', header=0, error_bad_lines=False)
#cup = pd.read_csv("data/cup/AREA_NORD_EST_CSV.csv", sep='|')
cup.shape()